In [1]:
from model.BERTNoiseDefend import BertForSequenceClassification
from model.RoBERTaNoiseDefend import RobertaForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from textattack.models.wrappers import (
    ModelWrapper,
    PyTorchModelWrapper,
    SklearnModelWrapper,
    HuggingFaceModelWrapper,
)


config = AutoConfig.from_pretrained("model/weights/bert-base-uncased-imdb")
tokenizer = AutoTokenizer.from_pretrained(
    "model/weights/bert-base-uncased-imdb", use_fast=True
)

model = BertForSequenceClassification(config)
state = AutoModelForSequenceClassification.from_pretrained(
    "model/weights/bert-base-uncased-imdb"
)

#config = AutoConfig.from_pretrained("textattack/bert-base-uncased-ag-news")
#tokenizer = AutoTokenizer.from_pretrained(
#    "textattack/bert-base-uncased-ag-news", use_fast=True
#)
#
#model = BertForSequenceClassification(config)
#state = AutoModelForSequenceClassification.from_pretrained(
#    "textattack/bert-base-uncased-ag-news"
#)
model.load_state_dict(state.state_dict())
model.to("cpu")
model.eval()
token_length = 256
tokenizer.model_max_length=token_length
tokens = tokenizer(["This film is terrible. You don't really need to read this review further. If you are planning on watching it, suffice to say - don't (unless you are studying how not to make a good movie).<br /><br />The acting is horrendous... serious amateur hour. Throughout the movie I thought that it was interesting that they found someone who speaks and looks like Michael Madsen, only to find out that it is actually him! A new low even for him!!<br /><br />The plot is terrible. People who claim that it is original or good have probably never seen a decent movie before. Even by the standard of Hollywood action flicks, this is a terrible movie.<br /><br />Don't watch it!!! Go for a jog instead - at least you won't feel like killing yourself.","This film is terrible. You don't really need to read this review further. If you are planning on watching it, suffice to say - don't (unless you are studying how not to make a good movie).<br /><br />The acting is horrendous... serious amateur hour. Throughout the movie I thought that it was interesting that they found someone who speaks and looks like Michael Madsen, only to find out that it is actually him! A new low even for him!!<br /><br />The plot is terrible. People who claim that it is original or good have probably never seen a decent movie before. Even by the standard of Hollywood action flicks, this is a terrible movie.<br /><br />Don't watch it!!! Go for a jog instead - at least you won't feel like killing yourself."],add_special_tokens=True,
            padding="max_length",
            max_length=token_length,
            truncation=True,
            return_tensors="pt")

output = model(tokens["input_ids"],tokens["token_type_ids"],tokens["attention_mask"])

/opt/conda/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-26 16:53:24.122758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
for layers in model.bert.encoder.layer:
    print(layers.attention_output.shape)

torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])
torch.Size([2, 256, 768])


In [3]:
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
import torch
from tqdm import tqdm
from utils.dataloader import load_train_test_imdb_data
import numpy as np
import datasets
num_repetitions = 3
batch=300
y_pred_BERT = []
train_data, test_data = load_train_test_imdb_data(
        "data/aclImdb"
    )
token_length = 256

#sst2_dataset = datasets.load_dataset("ag_news")
#train_data = sst2_dataset["train"]
#test_data = sst2_dataset["test"]

test_labels = np.array(test_data["label"])
bert_input = list(train_data["text"])
device = "cuda"
BERT = HuggingFaceModelWrapper(model,tokenizer)
std_features = [None] * len(model.bert.encoder.layer)
mean_features = [None] * len(model.bert.encoder.layer)
for i in tqdm(range(0,len(bert_input)//batch)):
    BERT(bert_input[i*batch:(i+1)*batch])
    for k in range(0,len(model.bert.encoder.layer)):
        if std_features[k] is None:
            std_features[k] = torch.std(model.bert.encoder.layer[k].attention_output.cpu(), dim=1)
            mean_features[k] = torch.mean(model.bert.encoder.layer[k].attention_output.cpu(), dim=1)
        else:
            std_features[k] = torch.cat((std_features[k], torch.std(model.bert.encoder.layer[k].attention_output.cpu(), dim=1)), 0)
            mean_features[k] = torch.cat((mean_features[k], torch.mean(model.bert.encoder.layer[k].attention_output.cpu(), dim=1)), 0)


100%|██████████| 83/83 [21:52<00:00, 15.81s/it]


In [5]:
std_features[-1][0]

tensor([0.7077, 0.6871, 0.6129, 0.4894, 0.7485, 0.7143, 0.5300, 0.6213, 0.6867,
        0.6139, 0.4979, 0.5418, 0.6302, 0.7276, 0.6055, 0.5985, 0.5686, 0.5637,
        0.6699, 0.5814, 0.6581, 0.4979, 0.6608, 0.8249, 0.5756, 0.6723, 0.6272,
        0.6377, 0.5875, 0.5100, 0.5784, 0.7499, 0.7780, 0.6422, 0.6850, 0.5305,
        0.7380, 0.6117, 0.6848, 0.7503, 0.5564, 0.5842, 0.6640, 0.7983, 0.6015,
        0.5398, 0.7789, 0.6039, 0.5873, 0.6397, 0.6443, 0.5636, 0.7471, 0.5026,
        0.5622, 0.8786, 0.6033, 0.5454, 0.5530, 0.5923, 0.6161, 0.5145, 0.5105,
        0.5772, 0.6860, 0.7123, 0.5608, 0.5917, 1.1347, 0.7233, 0.5703, 0.5986,
        0.6857, 0.5697, 0.6539, 0.6227, 0.6788, 0.5577, 0.5776, 0.6092, 0.6672,
        0.5477, 0.7048, 0.5766, 0.6855, 0.5324, 0.6712, 0.6928, 0.7110, 0.7450,
        0.9230, 0.7267, 0.6039, 0.6843, 0.5374, 0.5490, 0.5205, 0.5011, 0.5299,
        0.7129, 0.6245, 0.7136, 0.6475, 0.5297, 0.7072, 0.4490, 0.8685, 0.7627,
        0.6269, 0.6051, 0.6547, 0.8062, 

In [6]:
std_features

[tensor([[0.9790, 0.8445, 0.8675,  ..., 0.8474, 0.7660, 0.9136],
         [0.9070, 0.9158, 0.8144,  ..., 0.7567, 0.8066, 0.9238],
         [1.0075, 0.7432, 0.8451,  ..., 0.7434, 0.8283, 0.9944],
         ...,
         [1.0425, 0.7874, 0.8396,  ..., 0.7522, 0.8110, 0.9341],
         [0.9855, 0.7346, 0.8040,  ..., 0.8014, 0.8319, 1.0146],
         [0.8919, 0.9256, 0.7064,  ..., 0.7855, 0.8086, 0.8012]]),
 tensor([[0.9255, 0.8587, 0.7663,  ..., 0.7322, 0.6925, 0.9016],
         [0.8219, 0.8063, 0.7371,  ..., 0.6367, 0.6655, 0.9970],
         [0.9221, 0.7261, 0.7866,  ..., 0.6317, 0.6938, 0.9600],
         ...,
         [0.9414, 0.7305, 0.7569,  ..., 0.6611, 0.6999, 0.9592],
         [0.9391, 0.7184, 0.7201,  ..., 0.6618, 0.7316, 1.0245],
         [0.7227, 0.7092, 0.7418,  ..., 0.6525, 0.5743, 0.8499]]),
 tensor([[0.9307, 0.9139, 0.7352,  ..., 0.6902, 0.7027, 0.9382],
         [0.8528, 0.8126, 0.7538,  ..., 0.7255, 0.6938, 1.0781],
         [0.9087, 0.7793, 0.7588,  ..., 0.6775, 0.6815, 0.

In [12]:
import torch

def combine_standard_deviation_mean(std_devs, means,token_length):
    n = len(std_devs)
    if n == 0:
        return None, None
    mean = torch.mean(means, dim=0)
    q = torch.sum((token_length-1)*std_devs + token_length*torch.pow(means, 2),dim=0)
    std_dev = torch.sqrt( (q - (token_length*std_devs.shape[0])*torch.pow(mean, 2))/(token_length*std_devs.shape[0]-1) )
    return std_dev, mean

# sanity check
x = torch.rand((100*256,756))
std1=torch.std(x,dim=0)

x_2 = x.view((100,256,756))
std = torch.std(x_2, dim=1)
mean = torch.mean(x_2, dim=1)
std,_ = combine_standard_deviation_mean(std,mean,token_length)
print(torch.std(std1-std))

tensor(6.6726e-05)


In [13]:
print(std)

tensor([0.5362, 0.5366, 0.5353, 0.5358, 0.5379, 0.5371, 0.5365, 0.5371, 0.5372,
        0.5360, 0.5357, 0.5369, 0.5359, 0.5380, 0.5385, 0.5357, 0.5361, 0.5358,
        0.5369, 0.5357, 0.5364, 0.5367, 0.5372, 0.5368, 0.5359, 0.5363, 0.5370,
        0.5367, 0.5362, 0.5362, 0.5357, 0.5367, 0.5355, 0.5374, 0.5356, 0.5366,
        0.5365, 0.5365, 0.5359, 0.5357, 0.5369, 0.5352, 0.5372, 0.5366, 0.5366,
        0.5371, 0.5371, 0.5363, 0.5378, 0.5360, 0.5363, 0.5367, 0.5373, 0.5364,
        0.5369, 0.5364, 0.5359, 0.5369, 0.5362, 0.5367, 0.5355, 0.5354, 0.5360,
        0.5358, 0.5368, 0.5365, 0.5354, 0.5367, 0.5363, 0.5365, 0.5365, 0.5356,
        0.5371, 0.5369, 0.5353, 0.5373, 0.5349, 0.5363, 0.5358, 0.5364, 0.5366,
        0.5362, 0.5364, 0.5361, 0.5363, 0.5375, 0.5349, 0.5381, 0.5372, 0.5352,
        0.5357, 0.5352, 0.5360, 0.5357, 0.5362, 0.5362, 0.5363, 0.5377, 0.5360,
        0.5359, 0.5361, 0.5372, 0.5363, 0.5363, 0.5361, 0.5360, 0.5367, 0.5351,
        0.5366, 0.5363, 0.5361, 0.5363, 

In [14]:
out_list = []
for i in range(0,len(std_features)):
    std,mean = combine_standard_deviation_mean(std_features[i],mean_features[i],256)
    out_list.append(std)

tensor(0.9641)
tensor(0.9008)
tensor(0.8825)
tensor(0.8830)
tensor(0.8708)
tensor(0.8718)
tensor(0.8671)
tensor(0.8688)
tensor(0.8759)
tensor(0.8661)
tensor(0.8725)
tensor(0.8726)


In [10]:
#BERT_IMDB_STD_FEATURE_DIM = torch.cat(out_list)
torch.save(x, 'BERT_IMDB_STD_FEATURE_DIM.pt')


In [11]:
import math

import torch

def combine_standard_deviation(std_devs, means):
    n = len(std_devs)
    if n == 0 or torch.isnan(std_devs).any() or torch.isnan(means).any():
        return None
    mean = torch.mean(means)
    variance = torch.mean(std_devs ** 2 + means ** 2) - mean ** 2
    std_dev = torch.sqrt(variance + 1e-8)
    return std_dev